In [19]:
# Copyright (c) OpenMMLab. All rights reserved.
from pre_processing import preprocess
from post_processing import postprocess
from inference import inference_batch_imgs
import cv2
import numpy as np
import yaml
import argparse
from utils import import_class, build_session, prepare_data
import torch


import pickle
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt

import cv2
import threading
import queue
import time

from IPython.display import Video

In [20]:
# Read json file
json_path = r'D:\DATN\project\data\preprocessed_data\keypoints_NLA_WLASL\2000.json'

with open(json_path, 'r') as f:
  wlasl2000_json = json.load(f)

In [21]:
wlasl_class_list_path = r'D:\DATN\project\data\raw_data\WLASL\split\wlasl_class_list.txt'
with open(wlasl_class_list_path, mode='r') as f:
    lines = f.readlines()
    label_to_gloss = dict()
    label_to_gloss_array = []
    for line in lines:
        data = line.split('\t')
        label = int(data[0])
        gloss = data[1].rstrip()
        label_to_gloss[label] = gloss
        label_to_gloss_array.append(gloss)

## Offline prediction

In [22]:
## Config area
# onnx_file = r"D:\DATN\project\data_prepare\onnx_model\rtmw-dw-x-l_simcc-cocktail14_270e-384x288_20231122\end2end.onnx"
onnx_file = r"D:\DATN\project\data_prepare\onnx_model\rtmpose-l_simcc-ucoco_dw-ucoco_270e-256x192-4d6dfc62_20230728\20230831\rtmpose_onnx\rtmpose-l_simcc-ucoco_dw-ucoco_270e-256x192-4d6dfc62_20230728\end2end.onnx"
device = "cpu"

In [23]:
# Build session with ONNXRuntime
sess = build_session(onnx_file, device)
h, w = sess.get_inputs()[0].shape[2:]
model_input_size = (w, h)
print(w,h)

192 256


In [24]:
%cd "D:\DATN\project\Pose-based-WLASL"

D:\DATN\project\Pose-based-WLASL


In [25]:
# Load GCN model

# model_config_path = r'D:\DATN\project\Pose-based-WLASL\checkpoints\stgcnpp-27-rtmw_onehot_top1=53.09.yaml' 
model_config_path = r"D:\DATN\project\mycode\best_weight_GCN\ctr-gcn-27-rtml-256-192 top1=43.22.yaml"
weights_path = r"D:\DATN\project\mycode\best_weight_GCN\ctr-gcn-27-rtml-256-192 top1=43.22.pt"

with open(model_config_path) as f: 
    arg = yaml.load(f, Loader=yaml.FullLoader)
arg = argparse.Namespace(**arg)

arg.model_args['graph_args']['layout'] = 'keypoint-27'

In [26]:

Model = import_class(arg.model)
model = Model(**arg.model_args)


weights = torch.load(weights_path, map_location='cpu')

new_weights = {}
for key, value in weights.items():
  new_weights[key[7:]] = value

model.load_state_dict(new_weights)
model.eval()
print("Model eval")

Model eval


In [27]:
def predict_offline(video_path):
    print(video_path)
    cap = cv2.VideoCapture(video_path)

    stack_frame = []

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        resized_img, center, scale = preprocess(frame, model_input_size)
        stack_frame.append(resized_img)

    # Time for pose estimation
    stack_frame = np.array(stack_frame, dtype=np.float32)
    total_frames = stack_frame.shape[0] 
    print(f"{stack_frame.shape[0]} frames")
    # Stack frame shape:  (34, 256, 192, 3)
    
    
    start_time = time.time()
    # inference
    outputs = inference_batch_imgs(sess, stack_frame)
    # postprocessing
    keypoints, scores = postprocess(outputs, model_input_size, center, scale)
    pose_time = time.time() - start_time

    results = np.concatenate((keypoints, scores[:,:, None]), axis=2)
    
    # keypoints.shape = (56, 133, 2)
    # scores.shape = (56,133, )


    # Data-prepare
    data = prepare_data(results, num_keypoint=27)
    # results.shape: (3, 150, 27, 1)
    data = np.expand_dims(data, axis=0)
    
    # Predict 
    start_time = time.time()
    with torch.no_grad():
        data = torch.Tensor(data)
        predictions =  model(data)
    model_time = time.time() - start_time
    
    label = torch.argmax(predictions)
    gloss = label_to_gloss[int(label)]
    return  gloss, total_frames, pose_time, model_time  


In [28]:
# # Load keypoint
# # data_path = r"D:\DATN\project\data\preprocessed_data\keypoints-rtm-l-384-288\skeleton-rtm-l-384-288.pkl"
# data_path = "D:\DATN\project\data\preprocessed_data\keypoints-rtm-l-256-192\skeleton-rtm-l-256-192.pkl"

# with open(data_path,
#           mode='rb') as f:
#     wlasl_keypoints = pickle.load(f)

In [29]:
# print("Shape of a sample value: ", wlasl_keypoints[id])

In [30]:
import os
input_dirs = [
    r"D:\dataset\rgb_WLASL2000\WLASL2000\\",
              ]
paths = []
for dir_path in input_dirs:
    for root, _, fnames in os.walk(dir_path):
        for fname in fnames:     
            path1 = os.path.join(root, fname) 
            paths.append(path1)
            
paths.sort()

In [36]:
video_paths = paths[:2]
memory = []
predict_true = 0
for video_path in video_paths:
    gloss, total_frames, pose_time, model_time   = predict_offline(video_path)
    memory.append((total_frames, pose_time, model_time))
    video_id = os.path.basename(video_path).split('.')[0]
    true_label = wlasl2000_json[video_id]['action'][0]
    true_gloss = label_to_gloss[true_label]
    print(gloss ,'   ', true_gloss)
    if gloss == true_gloss:
        predict_true += 1

D:\dataset\rgb_WLASL2000\WLASL2000\\00295.mp4
21 frames
s     a
D:\dataset\rgb_WLASL2000\WLASL2000\\00333.mp4
52 frames
stomach     abdomen


In [37]:
# Time counting

memory = np.array(memory)

sum_up_memory = np.sum(memory, axis=0)

In [38]:
sum_up_memory

array([73.        ,  5.80179787,  0.39365339])

In [39]:
print("pose time per frame: ", sum_up_memory[1]/sum_up_memory[0])
print("model time per a inference: ", sum_up_memory[2]/sum_up_memory[0])


pose time per frame:  0.07947668310714094
model time per a inference:  0.0053925122300239456


In [17]:
import cv2
import time

# Real-time pose estimator 

# Define box and image sizes
box_width, box_height = 400, 400
image_width, image_height = 256, 256

# Initialize the webcam
cap = cv2.VideoCapture(0)  # 0 for default webcam

prev_frame_time = 0
new_frame_time = 0
# font which we will be using to display FPS 
font = cv2.FONT_HERSHEY_SIMPLEX 


while True:
  # Capture frame-by-frame
  ret, frame = cap.read()

  # Check if frame is read correctly
  if not ret:
    print("Error: Unable to capture frame")
    break

  # Get frame dimensions
  frame_height, frame_width, _ = frame.shape

  # Calculate top-left corner of the box
  top_left_x = max(0, (frame_width - box_width) // 2)
  top_left_y = max(0, (frame_height - box_height) // 2)

  # Draw the box on the frame
  cv2.rectangle(frame, (top_left_x, top_left_y), (top_left_x + box_width, top_left_y + box_height), (0, 255, 0), 2)

  # Extract the box region
  box_region = frame[top_left_y:top_left_y + box_height, top_left_x:top_left_x + box_width]

  # Resize the extracted region
  resized_image = cv2.resize(box_region, (image_width, image_height))

    # time when we finish processing for this frame 
  new_frame_time = time.time() 

  # Calculating the fps 
  fps = 1/(new_frame_time-prev_frame_time) 
  prev_frame_time = new_frame_time 
  fps = round(fps,3) 
  fps = str(fps) 

  # putting the FPS count on the frame 
  cv2.putText(frame, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA) 



  # Display the frame with box and resized image
  cv2.imshow('Webcam Capture', frame)
  cv2.imshow('Extracted and Resized', resized_image)

  # Press 'q' to quit
  if cv2.waitKey(1) == ord('q'):
    break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [18]:
# how many frames to skip while inferencing
# configuring a higher value will result in better FPS (frames per rate), but accuracy might get impacted
SKIP_FRAME_COUNT = 3

# analyse the video
def analyse_video(pose_detector, lstm_classifier, video_path):
    # open the video
    cap = cv2.VideoCapture(video_path)
    # width of image frame
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    # height of image frame
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # frames per second of the input video
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # total number of frames in the video
    tot_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # video output codec
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # extract the file name from video path
    file_name = ntpath.basename(video_path)
    # video writer
    vid_writer = cv2.VideoWriter('res_{}'.format(
        file_name), fourcc, 30, (width, height))
    # counter
    counter = 0
    
    
    # buffer to keep the output of detectron2 pose estimation
    buffer_window = []
    # start time
    start = time.time()    
    label = None
    # iterate through the video
    while True:
        # read the frame
        ret, frame = cap.read()
        # return if end of the video
        if ret == False:
            break
        # make a copy of the frame
        img = frame.copy()
        if(counter % (SKIP_FRAME_COUNT+1) == 0):             
          # predict pose estimation on the frame
          outputs = pose_detector(frame)          
          # filter the outputs with a good confidence score
          persons, pIndicies = filter_persons(outputs)
          if len(persons) >= 1:
              # pick only pose estimation results of the first person.
              # actually, we expect only one person to be present in the video. 
              p = persons[0]
              # draw the body joints on the person body
              draw_keypoints(p, img)
              # input feature array for lstm
              features = []
              # add pose estimate results to the feature array
              for i, row in enumerate(p):
                  features.append(row[0])
                  features.append(row[1])

              # append the feature array into the buffer
              # note that max buffer size is 32 and buffer_window operates in a sliding window fashion
              if len(buffer_window) < WINDOW_SIZE:
                  buffer_window.append(features)
              else:
                  # convert input to tensor
                  model_input = torch.Tensor(np.array(buffer_window, dtype=np.float32))
                  # add extra dimension
                  model_input = torch.unsqueeze(model_input, dim=0)
                  # predict the action class using lstm
                  y_pred = lstm_classifier(model_input)
                  prob = F.softmax(y_pred, dim=1)
                  # get the index of the max probability
                  pred_index = prob.data.max(dim=1)[1]
                  # pop the first value from buffer_window and add the new entry in FIFO fashion, to have a sliding window of size 32.
                  buffer_window.pop(0)
                  buffer_window.append(features)
                  label = LABELS[pred_index.numpy()[0]]
                  #print("Label detected ", label)
        
        # add predicted label into the frame
        if label is not None:
            cv2.putText(img, 'Action: {}'.format(label),
        (int(width-400), height-50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (102, 255, 255), 2)                  
        # increment counter
        counter += 1
        # write the frame into the result video                    
        vid_writer.write(img)
        # compute the completion percentage
        percentage = int(counter*100/tot_frames)
        # return the completion percentage
        yield "data:" + str(percentage) + "\n\n"
    analyze_done = time.time()
    print("Video processing finished in ", analyze_done - start)
